In [65]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

# Dataset Initialization

In [66]:
# # res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# import json
  
# # Opening JSON file
# f = open('./BigData.json')
  
# # returns JSON object as 
# # a dictionary
# res = json.load(f)
  
# # Iterating through the json
# # list
# # for i in res:
# #     print(i)
  
# # Closing file
# f.close()
# # text = res
# # text[:100]
# print(res)

We need this in a dataframe, which we build from the `text` string like so:

In [67]:
data = pd.read_json('./BigData.json')
data.head()

,ticket,next action
0,"{'Name': 'I-00000001', 'Notice Level': 'Middle...","[step 1: Update Notice Level to Middle, step 2..."
1,"{'Name': 'I-00000002', 'Notice Level': 'High',...","[step 1: Update Notice Level to High, step 2: ..."
2,"{'Name': 'I-00000003', 'Notice Level': 'Low', ...","[step 1: Update Notice Level to Low, step 2: C..."
3,"{'Name': 'I-00000004', 'Notice Level': 'Middle...","[step 1: Update Notice Level to Middle, step 2..."
4,"{'Name': 'I-00000094', 'Notice Level': 'High',...","[step 1: Update Notice Level to High, step 2: ..."


We will take all samples from `sentence_A` and build sentence embeddings for each - which we can then store in FAISS.

In [68]:
# sentences = data['ticket']['ticketDetail'].tolist()
# sentences[:5]
sentences = [] 
for i in data['ticket']:
    sentences.append(i['ticketDetail']) 
print(sentences) 

# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

['Detected suspicious network activity from an internal IP address.', 'Received a report of a data breach involving customer information.', "Identified a vulnerability in the company's web application.", 'Received multiple reports of phishing emails targeting employees.', "Received reports of a possible compromise of an executive's email account.", 'Detected a potential phishing attempt targeting employees via email.', "Received reports of a possible compromise of the company's file sharing system.", "Identified a critical vulnerability in the company's database management system.", 'Received reports of a suspicious physical device connected to a workstation.', "Detected suspicious activity in the company's network traffic logs."]


In [69]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))


Now we have 14.5K *unique* sentences, a much better size. We'll go ahead and build the sentence embeddings (this can take some time, feel free to download the embeddings from [here]()). TODO add link

In [70]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(10, 768)

We can save/load from file in the case of needing to reload the notebook for any reason later.

In [71]:
sentence_embeddings.shape[0]

10

In [72]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [73]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 11


We setup our FAISS database dimensionality (number of dimensions per vector) based on these vectors.

In [74]:
d = sentence_embeddings.shape[1]
d

768

# Flat L2 Index

We initialize the flat L2 distance index `IndexFlatL2`, all we need is the specify the vector dimensionality - which in this case is `d == 768` (to align with the sentence-BERT model output embeddings of size `768`).

In [75]:
index = faiss.IndexFlatL2(d)

Often, we will use indexes that require us to `train` them on our data before being used (if we are grouping or transforming the data in any way). `IndexFlatL2` however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector `xq` during search. So, in this case, no training is required - which we can confirm by checking the `is_trained` attribute.

In [76]:
index.is_trained

True

Okay so once we're happy that our index is prepared, we then add new vectors using the `add` method.

In [77]:
index.add(sentence_embeddings)

In [78]:
index.ntotal

10

Then search given a query `xq` and number of nearest neigbors to return `k`.

In [79]:
k = 4
xq = model.encode(["Received reports of a possible compromise of an executive's email account."])

In [80]:
%%time
D, I = index.search(xq, k)  # search
print(I, D)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[1 7 9 3]] [[3.3633932e-11 8.8316727e+01 1.0385466e+02 1.5351736e+02]]
CPU times: total: 0 ns
Wall time: 26.4 ms


Here we're returning indices `7460`, `10940`, `3781`, and `5747`, which returns:

In [81]:
[f'{i}: {sentences[i]}' for i in I[0]]



["1: Received reports of a possible compromise of an executive's email account.",
 "7: Received reports of a possible compromise of the company's file sharing system.",
 '9: Detected a potential phishing attempt targeting employees via email.',
 '3: Received a report of a data breach involving customer information.']

Clearly we have some good matches, everything returned includes people running with a football, or on the context of a football match. Now, if we'd rather extract the numerical vectors from FAISS, we can do that too.

In [82]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [83]:
vecs.shape

(4, 768)

In [84]:
vecs[0][:100]

array([-0.50518423,  0.48758391,  0.66647851,  0.03475399,  0.37630296,
       -0.11395732,  0.77897424,  0.37778905,  0.26803911, -0.15433937,
       -0.09677369,  0.2091168 ,  0.42357036,  0.61092204, -0.77611792,
       -0.35400027, -0.88501906,  0.38061816,  0.5002628 , -0.20286959,
       -0.40612507, -0.10444601, -0.46866933,  0.07575669,  0.55922788,
       -0.02680862,  0.05735995, -0.63364196,  0.10851849,  0.40746295,
       -0.43443438, -0.24623522,  0.11907054, -0.65765905, -0.11376578,
       -0.57217783,  0.18011299,  0.48327035, -0.36807847,  0.27443936,
       -0.76391739,  0.58233398,  0.71162719,  1.09151471, -0.81575686,
       -0.87680387, -0.05219297,  0.79401863,  0.16443107, -0.71088141,
       -0.85350603,  0.70048892,  1.17236733,  0.51407832, -0.95886159,
        0.50808734,  0.14186063, -1.46503985, -0.72266203,  0.70568979,
       -0.94487143, -0.68937463,  0.99473774,  0.51572955, -1.02725387,
        0.39197078,  0.11724776, -0.0588825 , -0.63245457, -0.53

---

# Adding Partitioning to the Index

FAISS allows us to add an additional step to optimize our search efficiency using a variety of different methods. A popular approach is to partition the index into *[Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram)*. Using this method we would take our query vector `xq`, identify the *cell* it belongs to, and then use our `IndexFlatL2` to search between the query vector `xq` and all indexed vectors belonging to that *cell*. We can also include vectors from other nearby cells too.

We initialize our new partitioned index by first adding our previous `IndexFlatL2` operation as a quantization step (another step in the search process), and feeding this into the new `IndexIVFFlat` operation like so:

In [85]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

Here we've added a new parameter `nlist`. We use `nlist` to define how many partitions we'd like our index to have. 

When we built the previous, `IndexFlatL2`-only index, we noted that no training was required as no grouping/transformation was required to build that index. Now that we've added partitioning using `IndexIVFFlat`, this is no longer the case. Let's take a look at the `is_trained` attribute.

In [86]:
index.is_trained

False

So, what we need to do now is `train` our index on our data, which we do *before* adding any data to the index (otherwise the index cannot know how to group each vector).

In [87]:
index.train(sentence_embeddings)
index.is_trained

RuntimeError: Error in void __cdecl faiss::Clustering::train_encoded(__int64,const unsigned char *,const struct faiss::Index *,struct faiss::Index &,const float *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\Clustering.cpp:281: Error: 'nx >= k' failed: Number of training points (10) should be at least as large as number of clusters (50)

Now our index is trained, we add our data just as we did before.

In [ ]:
index.add(sentence_embeddings)
index.ntotal

RuntimeError: Error in void __cdecl faiss::IndexIVFFlat::add_core(__int64,const float *,const __int64 *,const __int64 *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\IndexIVFFlat.cpp:46: Error: 'is_trained' failed

Let's search again using the same indexed sentence embeddings and the same query `xq`.

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[-1 -1 -1 -1]]
CPU times: total: 0 ns
Wall time: 12.4 ms


We can increase the number of nearby cells to search too with `nprobe`.

In [ ]:
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

Increasing the number of `nprobe` will improve the accuracy of our search, but cost time. Our earlier `IndexFlatL2`-only search was *exhaustive* (it compared every single vector) and so it identified the closest matches with a perfect accuracy. The smaller our `nprobe` value, the smaller scope that we search. We received perfect results (that matched our previous `IndexFlatL2`-only results - `7460`, `10940`, `3781`, `5747`), however, if we found that we were not getting closely matching results, we could simply bump `nprobe` up further - improving accuracy, but increasing time-taken too.

It's worth noting that the time taken can change with each run too, if we rerun the above block, we usually get a different time:

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

For IVF (and IMI) indexes, before attempting to use the `reconstruct` method, we need to call the `make_direct_map` method - otherwise we will return a `RunetimeError`.

In [ ]:
index.reconstruct(7460)

With `make_direct_map`:

In [ ]:
index.make_direct_map()

In [ ]:
index.reconstruct(7460)[:100]

We've now significantly reduced the search time, what can we do next?

# Quantization

Well, when storing these vectors we're storing the full (eg `Flat`) vector. Now in very big datasets this can quickly become a problem. Typically, we look at big datasets, and when working with large dataset we will find that storing the full vectors consumes too much space.

Fortunately, FAISS comes with the ability to *compress* our vectors using transformations based on *Product Quantization* (PQ). But, what is PQ? Well, we can view it as an additional approximation step similar to our use of **IVF**, which allowed us to *approximate* by reducing the scope of our search. PQ is slightly different however, and approximates the distance (or similarity) calculation instead.

[PQ explanation TODO REMOVE](https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/)

PQ achieves this approximated distance operation by compressing the vectors themselves. This consists of a few steps.

1. We split the every original vector into several subvectors.

2. For each set of subvectors, we perform a clustering operation, creating many centroids for each subvector set.

3. In our vector of subvectors, we replace each subvector with the ID of it's nearest centroid.

In [ ]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

Again, we'll need to `train` the index.

In [ ]:
index.is_trained

In [ ]:
index.train(sentence_embeddings)

And `add` our vectors.

In [ ]:
index.add(sentence_embeddings)

Let's compare it to our previous index *without* PQ, and an `nprobe` value of `10`.

In [ ]:
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

Through adding PQ we've reduced our search time from ~7.5ms to ~5ms, a small difference on a dataset of this size, but when scaled to larger datasets this can make a huge difference.

Now, we should also notice the slightly different results being returned. Beforehand with our exhaustive L2 search we were returning `7460`, `10940`, `3781`, and `5747`. Now, we see a slightly different order to our results - and two different vectors, `5013` and `5370`.

Each of our speed optimization operations, `IVF` and `PQ`, come at the cost of accuracy. Now, if we print out these results we will nonetheless find that each item is still a relevant match:

In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

So although we might not get the *perfect* result, we still get close - and thanks to the approximation, we get a significant speed boost